# Deep Learning with Python

## 8.1 Text generation with LSTM

> 使用 LSTM 生成文本

以前有人说过：“generating sequential data is the closest computers get to dreaming”，让计算机生成序列是很有魅力的事情。我们将以文本生成为例，探讨如何将循环神经网络用于生成序列数据。这项技术也可以用于音乐的生成、语音合成、聊天机器人对话生成、甚至是电影剧本的编写等等。

其实，我们现在熟知的 LSTM 算法，最早被开发出来的时候，就是用于逐字符地生成文本的。

### 序列数据的生成

用深度学习生成序列的通用方法，就是训练一个网络(一般用 RNN 或 CNN)，输入前面的 Token，预测序列中接下来的 Token。

说的术语化一些：给定前面的 Token，能够对下一个 Token 的概率进行建模的网络叫作「语言模型(language model)」。语言模型能够捕捉到语言的统计结构 ——「潜在空间(latent space)」。训练好一个语言模型，输入初始文本字符串（称为「条件数据」，conditioning data），从语言模型中采样，就可以生成新 Token，把新的 Token 加入条件数据中，再次输入，重复这个过程就可以生成出任意长度的序列。

我们从一个简单的例子开始：用一个 LSTM 层，输入文本语料的 N 个字符组成的字符串，训练模型来生成第 N+1 个字符。模型的输出是做 softmax，在所有可能的字符上，得到下一个字符的概率分布。这个模型叫作「字符级的神经语言模型」(character-level neural language model)。

![使用语言模型逐个字符生成文本的过程](https://tva1.sinaimg.cn/large/007S8ZIlgy1ghq58v9qd6j319c0ka41y.jpg)

### 采样策略

使用字符级的神经语言模型生成文本时，最重要的问题是如何选择下一个字符。这里有几张常用方法：

- 贪婪采样(greedy sampling)：始终选择可能性最大的下一个字符。这个方法很可能得到重复的、可预测的字符串，而且可能意思不连贯。（输入法联想）
- 纯随机采样：从均匀概率分布中抽取下一个字符，其中每个字符的概率相同。这样随机性太高，几乎不会生成出有趣的内容。（就是胡乱输出字符的组合）
- 随机采样(stochastic sampling)：根据语言模型的结果，如果下一个字符是 e 的概率为 0.3，那么你会有 30% 的概率选择它。有一点的随机性，让生成的内容更~~随意~~富有变化，但又不是完全随机，输出可以比较有意思。

随机采样看上去很好，很有创造性，但有个问题是无法控制随机性的大小：随机性越大，可能富有创造性，但可能胡乱输出；随机性越小，可能更接近真实词句，但太死板、可预测。

为了在采样过程中控制随机性的大小，引入一个参数：「softmax 温度」(softmax temperature)，用于表示采样概率分布的熵，即表示所选择的下一个字符会有多么出人意料或多么可预测：

- 更高的温度：熵更大的采样分布，会生成更加出人意料、更加无结构的数据；
- 更低的温度：对应更小的随机性，会生成更加可预测的数据。

![对同一个概率分布进行不同的重新加权：更低的温度=更确定，更高的温度=更随机](https://tva1.sinaimg.cn/large/007S8ZIlgy1ghq692zm1nj316f0u0k9r.jpg)

具体的实现是，给定 temperature 值，对模型的 softmax 输出重新加权，得到新的概率分布：

In [1]:
import numpy as np

def rewight_distribution(original_distributon, temperature=0.5):
    '''
    对于不同的 softmax 温度，对概率分布进行重新加权
    '''
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

### 字符级 LSTM 文本生成实现

理论就上面那些了，现在，我们要用 Keras 来实现字符级的 LSTM 文本生成了。

#### 数据准备

首先，我们需要大量的文本数据(语料，corpus)来训练语言模型。可以去找足够大的一个或多个文本文件：维基百科、各种书籍等都可。这里我们选择用一些尼采的作品（英文译本），这样我们学习出来的语言模型将是有尼采的写作风格和主题的。（插：我，我以前自己写野生模型玩，都是用鲁迅😂）

In [2]:
# 下载语料，并将其转换为全小写

from tensorflow import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt', 
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


606208/600901 [==============================] - 430s 709us/step
Corpus length: 600893


接下来，我们要把文本做成数据 (向量化)：从 text 里提取长度为 `maxlen` 的序列(序列之间存在部分重叠)，进行 one-hot 编码，然后打包成 `(sequences, maxlen, unique_characters)` 形状的。同时，还需要准备数组 `y`，包含对应的目标，即在每一个所提取的序列之后出现的字符(也是 one-hot 编码的)：

In [6]:
# 将字符序列向量化

maxlen = 60     # 每个序列的长度
step = 3        # 每 3 个字符采样一个新序列
sentences = []  # 保存所提取的序列
next_chars = [] # sentences 的下一个字符

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i+maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
char_indices = dict((char, chars.index(char)) for char in chars)
# 插：上面这两行代码 6
print('Unique characters:', len(chars))

print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


#### 构建网络

我们要用到的网络其实很简单，一个 LSTM 层 + 一个 softmax 激活的 Dense 层就可以了。其实并不一定要用 LSTM，用一维卷积层也是可以生成序列的。

In [7]:
# 用于预测下一个字符的单层 LSTM 模型

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

In [8]:
# 模型编译配置

from tensorflow.keras import optimizers

optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

#### 训练语言模型并从中采样

给定一个语言模型和一个种子文本片段，就可以通过重复以下操作来生成新的文本：

1. 给定目前已有文本，从模型中得到下一个字符的概率分布；
2. 根据某个温度对分布进行重新加权；
3. 根据重新加权后的分布对下一个字符进行随机采样；
4. 将新字符添加到文本末尾。

在训练模型之前，我们先把「采样函数」写了：

In [9]:
def sample(preds, temperature=1.0):
    '''
    对模型得到的原始概率分布重新加权，并从中抽取一个字符索引
    '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

最后，再来训练并生成文本。我们在每轮完成后都使用一系列不同的温度值来生成文本，这样就可以看到，随着模型收敛，生成的文本如何变化，以及温度对采样策略的影响：

In [17]:
# 文本生成循环

import random

for epoch in range(1, 60):    # 训练 60 个轮次
    print(f'👉\033[1;35m epoch {epoch} \033[0m')    # print('epoch', epoch)
    
    model.fit(x, y,
              batch_size=128,
              epochs=1)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print(f'  📖 Generating with seed: "\033[1;32;43m{generated_text}\033[0m"')    # print(f' Generating with seed: "{generated_text}"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print(f'\n   \033[1;36m 🌡️ temperature: {temperature}\033[0m')    # print('\n  temperature:', temperature)
        print(generated_text, end='')
        for i in range(400):    # 生成 400 个字符
            # one-hot 编码目前有的文本
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
            
            # 预测，采样，生成下一字符
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            print(next_char, end='')
            
            generated_text = generated_text[1:] + next_char
            
    print('\n' + '-' * 20)

👉 epoch 1 
1565/1565 [==============================] - 170s 108ms/step - loss: 1.4089
  📖 Generating with seed: "ary!--will at least be entitled to demand in return that
psy"

    🌡️ temperature: 0.2
ary!--will at least be entitled to demand in return that
psychological senses of the the most comprehensed that is a sense of a perhaps the experience of the heart the present that the profound that the experience of the exploition and present that is a self and the present that is a more of the senses of a more and the sense of a more art and the the exploition and self-contempt of a perhaps the superiom and perhaps the contempt of the superiom and all th
    🌡️ temperature: 0.5
superiom and perhaps the contempt of the superiom and all the instance and plays place of comprehensed and so in morals and all the auther to present mettoral of the senses and fellines to have the conceive that the
possibility of the expendeness, the hasters as how to really expendened and
all that the forenies;

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


sant, who commanent
as those whom them has 'maves for pressonce of list as truth," they are a matter the name is, "that it is noth
    🌡️ temperature: 1.2
t as truth," they are a matter the name is, "that it is nothing, but," all virtuan as his world suxter? is his feetering refuin finger, that is not: its god, als quieus' has
far as ruse is atmashfof, onedy fedufined? will assumphit, "ye willbum'." like
the truiloso
inlogged,--this haid men (personar " logic will"
opinive conectity. at us a
awmetting somewudes ajot, has
being dit wet
beegous. through which only he want clears its sound stands of wishes.


8
--------------------
👉 epoch 19 
1565/1565 [==============================] - 416s 266ms/step - loss: 1.2955
  📖 Generating with seed: "ditioned
(of the metaphysical world) and the world known to "

    🌡️ temperature: 0.2
ditioned
(of the metaphysical world) and the world known to the sense of the sense of the spirit and spirits of the sense of the sense of the best and soul the p

利用更多的数据训练更大的模型，训练时间更长，生成的样本会更连贯、更真实。但是，这样生成的文本并没有任何意义。机器所做的仅仅是从统计模型中对数据进行采样，它并没有理解人类的语言，也不知道自己在说什么。